# 1 - Relational Data Models

## ToDo

1. Breakdown into BCNF & display lossless retrieval
2. Do op's covered in complex_queries to reformatted database

Things to look into (potenitally relevant for sprint)
- Closure algo (req'd): Discover FD's ***
- Projection algo: 'Projecting' FD's onto subsets of R (used for decomp?)
- Decomp algo: Eliminates anomalies/risk of errors when working w db (req'd)
- Chase test: Test to ensure lossless decomp.

Should do this for all tables in db

### 1. BCNF

- [ ] Functional dependencies
- [ ] Closure
- [ ] Projection of functional dependences
- [ ] Keys, superkeys
- [ ] BCNF
- [ ] Lossless joins
- [ ] Relational algebra; express constraints in terms of relational algebra

### Functional Dependencies

#### For Main Tables:
- `SessionID` $\rightarrow$ `SessionID`
- `PortfolioID` $\rightarrow$ `PortfolioID`, `TotalAmt`, `Risk`
- `AllocID` $\rightarrow$ `AllocID`, `Ticker`, `Amount`
- `StockID` $\rightarrow$ `StockID`, `Ticker`, `Sector`, `Price`, `SD`, `ERet`
- `HistoryID` $\rightarrow$ `HistoryID`, `Ticker`, `Date`, `Price`
- (`PortfolioID`, `StockID`) $\rightarrow$ `PortfolioID`, `StockID`
- (`AllocID`, `StockID`) $\rightarrow$ `AllocID`, `StockID`
- (`StockID`, `HistoryID`) $\rightarrow$ `StockID`, `HistoryID`
- (`SessionID`, `PortfolioID`) $\rightarrow$ `SessionID`, `PortfolioID`
- (`PortfolioID`, `AllocID`) $\rightarrow$ `PortfolioID`, `AllocID`

This concise summary lists all functional dependencies derived from your table definitions and their respective primary keys.

### 2. Complex queries

- Go over selection of examples from class; include relational algebra which describes each operation